In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [77]:
import numpy as np
!pip install matplotlib
import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

In [78]:
action_hit = 0
action_strike = 1
actions = [action_hit, action_strike]
player_policy = np.zeros((22, 1), dtype=np.int)
# for i in range(12, 20):
#player_policy[i] = action_hit
# print(player_policy)
player_policy[20] = action_strike
player_policy[21] = action_strike


def target_policy_player(usable_ace_player,player_sum, dealer_card):
    return player_policy[player_sum]


def behaviour_policy_player():
    if np.random.binomial(1, 0.5) == 1:
        return action_strike
    return action_hit


dealer_policy = np.zeros(22)
for i in range(17, 22):
    dealer_policy[i] = action_strike


def get_card():

    card = np.minimum(np.random.randint(1, 14), 10)
    return card


def card_value(card):
    if card == 1:
        return 11
    else:
        return card


def play(player_policy, initial_state=None, initial_action=None):
    player_sum = 0
    player_t = []
    usable_ace_player = False

    dealer_card1 = 0
    dealer_card2 = 0
    usable_ace_dealer = False

    if initial_state is None:
        while player_sum < 12:
            card = get_card()
            player_sum += card_value(card)
            if player_sum > 21:
                if player_sum == 22:
                    player_sum -= 10
            else:
                usable_ace_player |= (1 == card)
        dealer_card1 = get_card()
        dealer_card2 = get_card()
    else:
        usable_ace_player, player_sum, dealer_card1 = initial_state
        dealer_card2 = get_card()

    state = [usable_ace_player, player_sum, dealer_card1]
    dealer_sum = card_value(dealer_card1) + card_value(dealer_card2)
    usable_ace_dealer = 1 in (dealer_card1, dealer_card2)
    if dealer_sum > 21:
        if dealer_sum == 22:
            dealer_sum -= 10
    assert dealer_sum <= 21
    assert player_sum <= 21

    while True:
        if initial_action != None:
            action = initial_action
            initial_action = None
        else:
            action = player_policy(usable_ace_player, player_sum, dealer_card1)

        player_t.append(
            [(usable_ace_player, player_sum, dealer_card1), action])
        if action == action_strike:
            break
        card = get_card()
        ace_count = usable_ace_player
        if card == 1:
            ace_count += 1
        player_sum += card_value(card)
        while player_sum > 21 and ace_count:
            player_sum -= 10
            ace_count -= 1
        if player_sum > 21:
            return state, -1, player_t
        assert player_sum <= 21
        usable_ace_player = (ace_count == 1)

    while True:
        action = dealer_policy[dealer_sum]
        if action == action_strike:
            break
        new_card = get_card()
        ace_count = int(usable_ace_dealer)
        if new_card == 1:
            ace_count += 1
        dealer_sum += card_value(new_card)
        while dealer_sum > 21 and ace_count:
            dealer_sum -= 10
            ace_count -= 1
        if dealer_sum > 21:
            return state, 1, player_t
        usable_ace_dealer = (ace_count == 1)

    assert player_sum <= 21 and dealer_sum <= 21
    return_ans = -1
    if player_sum > dealer_sum:
        return_ans = 1

    elif player_sum == dealer_sum:
        return_ans = 0

    return state, return_ans, player_t


def monte_carlo_off_policy(episodes):
    initial_state = [True, 13, 2]
    rhos = []
    returns = []
    for i in range(0, episodes):
        state, reward, player_trajectory = play(
            behaviour_policy_player, initial_state=initial_state)
        num = 1.0
        den = 1.0
        for (usable_ace, player_sum, dealer_card), action in player_trajectory:
            if action == target_policy_player(usable_ace, player_sum, dealer_card):
                den *= 0.5
            else:
                num = 0.0
                break

        rho = num/den
        returns = np.asarray(rhos)
        w_returns = np.multiply(rhos, returns)
        w_returns = np.cumsum(w_returns)
        rhos = np.cumsum(rhos)
        ordinary_sampling = w_returns/np.arange(1, episodes + 1)
        with np.errstate(divide='ignore', invalid='ignore'):
            weighted_sampling = np.where(rhos != 0, w_returns/rhos, 0)

        return ordinary_sampling, weighted_sampling


def monte_carlo_es(episodes):
    state_action_values = np.zeros((10, 10, 2, 2))
    state_action_pair_count = np.ones((10, 10, 2, 2))

    def behavior_policy(usable_ace, player_sum, dealer_card):
        usable_ace = int(usable_ace)
        player_sum -= 12
        dealer_card -= 1
        values_ = state_action_values[player_sum, dealer_card, usable_ace, :] / \
            state_action_pair_count[player_sum, dealer_card, usable_ace, :]
        return np.random.choice([action_ for action_, value_ in enumerate(values_) if value_ == np.max(values_)])


    for episode in (range(episodes)):
        initial_state = [bool(np.random.choice([0, 1])),
                         np.random.choice(range(12, 22)),
                         np.random.choice(range(1, 11))]
        initial_action = np.random.choice(actions)
        current_policy = behavior_policy if episode else target_policy_player
        state, reward, trajectory = play(
            current_policy, initial_state, initial_action)
        first_visit_check = set()
        for (usable_ace, player_sum, dealer_card), action in trajectory:
            usable_ace = int(usable_ace)
            player_sum -= 12
            dealer_card -= 1
            state_action = (usable_ace, player_sum, dealer_card, action)
            if state_action in first_visit_check:
                continue
            first_visit_check.add(state_action)

            state_action_values[player_sum, dealer_card,
                                usable_ace, action] += reward
            state_action_pair_count[player_sum,
                                    dealer_card, usable_ace, action] += 1

    return state_action_values / state_action_pair_count


def monte_carlo_on_policy(episodes):
    states_usable_ace = np.zeros((10, 10))
    states_usable_ace_count = np.ones((10, 10))
    states_no_usable_ace = np.zeros((10, 10))
    states_no_usable_ace_count = np.ones((10, 10))
    for i in range(0, episodes):
        state, reward, player_trajectory = play(target_policy_player)
        for (usable_ace, player_sum, dealer_card), action in player_trajectory:
            player_sum -= 12
            dealer_card -= 1
            if usable_ace:
                states_usable_ace_count[player_sum, dealer_card] += 1
                states_usable_ace[player_sum, dealer_card] += reward
            else:
                states_no_usable_ace_count[player_sum, dealer_card] += 1
                states_no_usable_ace[player_sum, dealer_card] += reward
    return states_usable_ace / states_usable_ace_count, states_no_usable_ace / states_no_usable_ace_count


In [79]:
print("Generating Figure 5.1....")
states_usable_ace_1, states_no_usable_ace_1 = monte_carlo_on_policy(10000)
states_usable_ace_2, states_no_usable_ace_2 = monte_carlo_on_policy(500000)
states = [states_usable_ace_1,states_usable_ace_2,states_no_usable_ace_1,states_no_usable_ace_2]
titles = ['Usable Ace, 10000 Episodes','Usable Ace, 500000 Episodes','No Usable Ace, 10000 Episodes','No Usable Ace, 500000 Episodes']
x, axes = plt.subplots(2,2,figsize=(40,30))
plt.subplots_adjust(wspace=0.1, hspace=0.2)
axes = axes.flatten()
for state, title, axis in zip(states,titles,axes):
    fig = sns.heatmap(np.flipud(state), cmap="YlGnBu", ax=axis, xticklabels=range(1, 11),
                          yticklabels=list(reversed(range(12, 22))))
    fig.set_ylabel('player sum', fontsize=30)
    fig.set_xlabel('dealer showing', fontsize=30)
    fig.set_title(title, fontsize=30)


In [80]:
plt.show()


Code for generating the figures in 6.2:

In [81]:
values = np.zeros(7)
values[1:6] = 0.5


0 is the left terminal state and 6 is the right terminal state.
states from 1 to 5 are non-terminal.

In [82]:
values[6] = 1
true_value = np.zeros(7)
true_value[1:6] = np.arange(1,6)/6.0
true_value[6] = 1
print(true_value)

In [83]:
right = 0
left = 1

In [84]:
def temporal_difference(values, alpha = 0.1, batch = False):
    state = 3
    trajectory = [state]
    rewards = [0]
    while True:
        prev_state = state
        if np.random.binomial(1,0.5)==left:
            state-=1
        else:
            state+=1
        reward = 0
        trajectory.append(state)
        if not batch:
            values[prev_state]+= alpha* (reward+values[state]-values[prev_state])
        if state == 6:
           # print("Terminal state")
            break
        if state ==0:
            break
        rewards.append(reward)
    return trajectory,rewards

In [85]:
def monte_carlo(vals, alpha = 0.1, batch = False):
    state = 3
    trajectory = [3]
    while True:
        if np.random.binomial(1,0.5)==left:
            state-=1
        else:
            state+=1
        trajectory.append(state)
        if state == 6:
            returns = 1.0
            #print("Terminal state")
            break
        elif state == 0:
            returns = 0.0
            break
            
    if batch == False:
        for state in trajectory[:-1]:
            values[state] += alpha * (returns - values[state])
    return trajectory, [returns] * (len(trajectory)-1)

        

In [86]:
def compute_state_value():
    plt.figure(1)
    episodes = [0,1,10,100]
    current_values = np.copy(values)
    for i in range(episodes[-1]+1):
        if i in episodes:
            plt.plot(current_values, label= str(i) + 'episodes')
        temporal_difference(current_values)
    plt.plot(true_value, label = 'true values')
    plt.xlabel('state')
    plt.ylabel('estimated values')


In [87]:
td_alphas = [0.15,0.1,0.05]
mc_alphas = [0.01,0.02,0.03,0.04]


In [88]:
def rms_error(td_alphas, mc_alphas):
    runs = 100
    eps = 101
    for i, alpha in enumerate(td_alphas + mc_alphas):
        total_errors = np.zeros(eps)
        if i < len(td_alphas):
            method = 'Temporal Difference'
            linestyle = 'solid'
        else:
            method = 'Monte Carlo'
            linestyle = 'dashdot'
        for r in range(runs):
            errors = []
            current_vals = np.copy(values)
            for i in range(eps):
                error = np.sqrt(np.sum(np.power(true_value-current_vals,2)))
                errors.append(error)
                if method == 'Temporal Difference':
                    temporal_difference(current_vals, alpha = alpha)
                else:
                    monte_carlo(current_vals, alpha = alpha)
            total_errors += np.asarray(errors)
        total_errors /= runs
        plt.plot(total_errors,linestyle = linestyle, label = method + ', alpha = %.02f' % alpha)
    plt.xlabel('eps')
    plt.ylabel('rms')
    

In [90]:
plt.figure(figsize = (10,20))
plt.subplot(2,1,1)
compute_state_value()
plt.subplot(2,1,2)
rms_error(td_alphas, mc_alphas)
plt.tight_layout
plt.show()